# Postgres Demo notebook

In this Jupyter notebook we will show an example of adding an additional connector to retrieve metadata from another source.

Notebooks allow us to mix explanatory text and diagrams with functional code (in this case Python). All the code you see is executing in a real environment. Read the [readme](../read-me-first.ipynb) to understand more.

This notebook isn't yet included in our demos by default, but we hope to include more on connectors.

Before running this notebook, you MUST run:
* [configuration notebook](../egeria-server-config.ipynb). 

It's also recommended you read through, and run the following notebooks, to get more background:
* [Building a Data Catalog](../asset-management-labs/building-a-data-catalog.ipynb)
* [Automated Curation](../asset-management-labs/automated-curation.ipynb)


## Infrastructure

These notebooks have been [deployed using Kubernetes](https://egeria-project.org/guides/operations/kubernetes/). We have used a [container](https://quay.io/repository/odpi/egeria-database-connectors) which includes the [Egeria Postgres connector](https://github.com/odpi/egeria-database-connectors) in addition to base Egeria.


First we'll check the existing servers are already started. (They must be configured)



In [7]:
# Standard checks for notebooks - ensure all are running, define standard functions
%run ../common/environment-check.ipynb


Checking OMAG Server Platform availability...
    Core Platform is active
        Checking OMAG Server cocoMDS2
           ... cocoMDS2 is configured
           ... cocoMDS2 is active - ready to begin
        Checking OMAG Server cocoMDS3
           ... cocoMDS3 is configured
           ... cocoMDS3 is active - ready to begin
        Checking OMAG Server cocoMDS5
           ... cocoMDS5 is configured
           ... cocoMDS5 is active - ready to begin
        Checking OMAG Server cocoMDS6
           ... cocoMDS6 is configured
           ... cocoMDS6 is active - ready to begin
    Data Lake Platform is active
        Checking OMAG Server cocoMDS1
           ... cocoMDS1 is configured
           ... cocoMDS1 is active - ready to begin
        Checking OMAG Server cocoMDS4
           ... cocoMDS4 is configured
           ... cocoMDS4 is active - ready to begin
        Checking OMAG Server cocoView1
           ... cocoView1 is configured
           ... cocoView1 is active - ready to begin


Now we are going to setup some standard values that are normally setup elsewhere - some code refactoring of the notebooks is needed to fully integrate.

In [8]:
#language requirement
import os

#admin operations - where/who
adminUserId     = "garygeeke"
devPlatformURL       = os.environ.get('devPlatformURL', 'https://localhost:9445')
devPlatformName      = "Dev Platform"
adminPlatformURL     = devPlatformURL
adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

# Event infrastructure & cohort membership
eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
jsonContentHeader = {'content-type':'application/json'}
eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

# Default metadata repository options
inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

# Pick up which repo type to use from environment if set, otherwise default to inmemory 
metadataRepositoryType   = os.environ.get('repositoryType', inMemoryRepositoryOption)



## Integration Connector

In this notebook we have added an example of using an [Egeria integration connector](https://egeria-project.org/concepts/integration-connector/?h=integration+connector)

![Diagram](https://egeria-project.org/connectors/integration/integration-connectors.svg)

Once started, this connector will retrieve metadata from PostGres. For example this will include:
    - tables
    - columns
    - databases
    
The metadata will then be mapped to [Egeria types](https://egeria-project.org/types/), and pushed into one of the Egeria repositories



## Integration 

We now need to configure the integration server & connector, and point to Postgres from where we will import metadata. More info about this can be found in the [connector readme](https://github.com/odpi/egeria-database-connectors#readme)

![Integration](https://egeria-project.org/concepts/integration-daemon.svg)





## Setup a new metadata server (with repository)

Here we will configure an additional metadata server. This will host both a repository, as well as act as a metadata access point.

See https://egeria-project.org/services/omis/database-integrator/overview/ for more info on the required service to support database integration.

This metadata server will join the 'dev' [cohort](https://egeria-project.org/services/omrs/cohort/?h=cohort) alongside cocoMDSx. 

This will mean it sends events to, and receives events from, other members of the cohort, including the integration server we will add later.

![Coco Cohorts](../images/coco-pharmaceuticals-systems-cohorts.png)

### Required OMASs

Egeria has a grouping of APIs known as the [Open Metadata Access Services](https://egeria-project.org/services/omas/) or OMAS.

The [Data Manager OMAS](https://egeria-project.org/services/omas/data-manager/overview/) is needed to support database integration, so we will enable it in the configuration below.

In [23]:

mdrServerName = 'dbisrv'
mdrServerPlatform = corePlatformURL

# Configure some of the basics (not all) including the required service to support database integration
configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)

# Enable the data-manager service, which is needed to support database integration
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, "data-manager", {})
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)

# We're going to join this server to the 'coco' cohort
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

# We'll now deploy the created configuration, and start the server
deployServerToPlatform(adminPlatformURL, adminUserId, mdrServerName, corePlatformURL)
activateServerOnPlatform(mdrServerName, "core platform", corePlatformURL)



   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/server-url-root?url=https://lab-core:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the data-manager access service for this server...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/access-services/data-manager )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the metadata repository...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/local-repository/mode/in-memory-repository )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the event bus for this server...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/event-bus )
   ...... Response:  {'class': 'VoidRespo

True

## Setup an integration server

We are going to set up a new [integration daemon](https://egeria-project.org/concepts/integration-daemon/) that will run our Postgres connector.

This can run a number of different integration services, which we refer to as our [Open Metadata Integration Services](https://egeria-project.org/services/omis/) (OMIS)

![OMIS](https://egeria-project.org/concepts/integration-daemon-in-action.svg)

This will make use of the [Database Integrator](https://egeria-project.org/services/omis/database-integrator/overview/?h=database+integrator) Open Metadata Integration Service (OMIS). 

In [83]:
# We'll setup a new integration server on the core platform
intServerName   = "intserv"
intServerPlatform=corePlatformURL
intServerPlatformURL=corePlatformURL
intServerPlatformName = "Core platform"

# Config information needed for the postgres connector
# TODO - this should be a server NPA - but this causes cohort issues with security connector, so using garygeeke for now
pgConnectorUser="garygeeke"
pgConnectorName="postgresql"
pgConnectorClass="org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider"
integrationServiceURLMarker="database-integrator"

# Authentication and URL for postgres - in a final lab this would be stored in secrets
pgUser="employees"
pgPass="r-b;=BB|-VRl}=<|>ATSqkR_"
pgURL="jdbc:postgresql://16c8f626-eu-gb.lb.appdomain.cloud:5432/employees"

# Setup some of the basics
configurePlatformURL(adminPlatformURL, adminUserId, intServerName, intServerPlatform)
configureDefaultAuditLog(adminPlatformURL, adminUserId, intServerName)




   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/intserv/server-url-root?url=https://lab-core:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the default audit log...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/intserv/audit-log-destinations/default )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}


### Connector properties

In the request body we configure properties needed by the connector. The values were set earlier. 

Egeria stores configuration data in encrypted files by default -- and this is pluggable. For example IBM stores configurations along with other cloud data (In cloudant).

In [84]:
# request body needed to setup the postgres connector
integrationServiceOptions={}
connectorConfigs = [
         {
        "class": "IntegrationConnectorConfig",
        "connectorName": "postgresql",
        "connectorUserId": pgConnectorUser,
        "connection":
        {
          "class": "Connection",
          "userId": pgUser,
          "clearPassword": pgPass,
          "connectorType":
          {
            "class": "ConnectorType",
            "connectorProviderClassName": pgConnectorClass
          },
          "endpoint":
          {
            "class": "Endpoint",
            "address": pgURL
          },
          "recognizedConfigurationProperties": [
            "url",
            "ssl"
          ],
          "configurationProperties":
          {
            "url": pgURL,
            "ssl": "false"
          }
        },
        "metadataSourceQualifiedName": pgConnectorName,
        "refreshTimeInterval": "3456",
        "usesBlockingCalls": "false",
        "permittedSynchronization": "FROM_THIRD_PARTY"
      }
    ]

# Setup the integration service, and then deploy the server
configureIntegrationService(adminPlatformURL, adminUserId, intServerName, mdrServerName, 
                                mdrServerPlatform, integrationServiceURLMarker, integrationServiceOptions, 
                                connectorConfigs)

deployServerToPlatform(adminPlatformURL, adminUserId, intServerName, corePlatformURL)



   ... configuring the database-integrator integration service in intserv integration daemon ...
{'class': 'IntegrationServiceRequestBody', 'omagserverPlatformRootURL': 'https://lab-core:9443', 'omagserverName': 'dbisrv', 'integrationServiceOptions': {}, 'integrationConnectorConfigs': [{'class': 'IntegrationConnectorConfig', 'connectorName': 'postgresql', 'connectorUserId': 'garygeeke', 'connection': {'class': 'Connection', 'userId': 'employees', 'clearPassword': 'r-b;=BB|-VRl}=<|>ATSqkR_', 'connectorType': {'class': 'ConnectorType', 'connectorProviderClassName': 'org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider'}, 'endpoint': {'class': 'Endpoint', 'address': 'jdbc:postgresql://16c8f626-eu-gb.lb.appdomain.cloud:5432/employees'}, 'recognizedConfigurationProperties': ['url', 'ssl'], 'configurationProperties': {'url': 'jdbc:postgresql://16c8f626-eu-gb.lb.appdomain.cloud:5432/employees', 'ssl': 'false'}}, 'metadataSourceQualifiedName': 'postgresql', '

## Starting the integration

We will now start the integration daemon, which will launch our postgres connector, and start pulling in metadata from postgres

Before executing this cell, you may want to [launch the ecosystem UI](https://79c191f6-eu-gb.lb.appdomain.cloud:8091/coco) to see what we have (very little),
for example by using Repository Explorer to view all entities of type RelationalColumn

You could also view the postgres database - For example using a [local install of pgweb](http://localhost:8081).

In [85]:
activateServerOnPlatform(intServerName, "core platform", corePlatformURL)

                Starting server intserv ...


True

## checking the integration service is running

In [87]:
# Get status of integration connector
intServerPlatformName = "Core platform"
intServerPlatformURL=corePlatformURL


getIntegrationDaemonStatus(intServerName, intServerPlatformName, intServerPlatformURL, adminUserId)

One integration service defined for integration daemon intserv
 
Integration Service: Database Integrator OMIS
  Integration Connector Reports: 
      Integration Connector: postgresql
         status:                   FAILED
         failingExceptionMessage:  POSTGRES-CONNECTOR-400-003 The Method PostgresConnector.refresh received an Invalid parameter exception from the OMAS server.
 


## Checking the metadata that is loaded

On the sample 'employees' database the process of loading the metadata will start. Once complete, it will then
enter a poll loop & periodically check for updates. 

![Integration](https://egeria-project.org/guides/developer/integration-connectors/integration-connector-methods.svg)

Once loaded we'd expect to see metadata like:

![Model](https://egeria-project.org/services/omas/data-manager/relational-database-model.svg)

These are the actual types:

![Types](https://egeria-project.org/services/omas/data-manager/relational-database-open-metadata-types.svg)

Now let's explore the UI.

Start by searching on cocoMDS2 for all assets of type 'RelationalTable' - employees is towards the end. Navigate from there

# STOP - Experimental area

The cells below are still being developed. Read through the text and comments carefully

## Extending the User Interface Configuration

These functions are copied through from the configuration notebook. The payload has been
modified to add the servers created above into type explorer, repository explorer, and Dino

**NOTE**: Currently this does not seem to work. However this will not affect the API, and since the new metadata server IS part of 'cocoCohort', federated queries to one of the other servers
such as cocoMDS2 will work fine - via UI or API



In [14]:

def configureGovernanceSolutionViewService(adminPlatformURL, adminUserId, viewServerName, viewService, remotePlatformURL,remoteServerName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + viewService + " Governance Solution View Service for this server...")
    url = adminCommandURLRoot + viewServerName + '/view-services/' + viewService
    jsonContentHeader = {'content-type':'application/json'}
    viewBody = {
        "class": "ViewServiceConfig",
        "omagserverPlatformRootURL": remotePlatformURL,
        "omagserverName" : remoteServerName
    }
    postAndPrintResult(url, json=viewBody, headers=jsonContentHeader)
    
def configureIntegrationViewService(adminPlatformURL, adminUserId, viewServerName, viewService, configBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + viewService + " Integration View Service for this server...")
    url = adminCommandURLRoot + viewServerName + '/view-services/' + viewService
    jsonContentHeader = {'content-type':'application/json'}
    postAndPrintResult(url, json=configBody, headers=jsonContentHeader)


# A view server supports the presentation server UI (a node based app). Here we run it on the datalake platform    
viewServerName          = "cocoView1"
viewServerUserId        = "cocoView1npa"
viewServerPassword      = "cocoView1passw0rd"
viewServerPlatform      = dataLakePlatformURL
viewServerType          = "View Server"

# Extra config already in config notebook
maxPageSize = "100"
organizationName = "Coco Pharmaceuticals"
serverSecurityConnectionBody = {
    "class": "Connection",
    "connectorType": {
        "class": "ConnectorType",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}
lakePlatformURL       = os.environ.get('devPlatformURL', 'https://localhost:9445')
lakePlatformName      = "Dev Platform"

# Configuration is similar to most servers
print("Configuring " + viewServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, viewServerName, viewServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, viewServerName)
configureServerType(adminPlatformURL,adminUserId,viewServerName,viewServerType)
configureOwningOrganization(adminPlatformURL, adminUserId, viewServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, viewServerName, viewServerUserId)
configurePassword(adminPlatformURL, adminUserId, viewServerName, viewServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, viewServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, viewServerName, eventBusBody)
configureDefaultAuditLog(adminPlatformURL, adminUserId, viewServerName)

# The governance solution view services currently only consist of glossary author
print ("Configuring the Governance Solution View Services")
remotePlatformURL=corePlatformURL
remoteServerName="cocoMDS2"
viewService="glossary-author"
configureGovernanceSolutionViewService(adminPlatformURL, adminUserId, viewServerName, viewService, remotePlatformURL,remoteServerName)

print ("Configuring the Integration View Services")

# repository explorer integration view service
viewService="rex"
rexConfigBody = {
            "class":"IntegrationViewServiceConfig",
            "viewServiceAdminClass":"org.odpi.openmetadata.viewservices.rex.admin.RexViewAdmin",
            "viewServiceFullName":"Repository Explorer",
            "viewServiceOperationalStatus":"ENABLED",
            "omagserverPlatformRootURL": "UNUSED",
            "omagserverName" : "UNUSED",
            "resourceEndpoints" : [
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "Core Platform",
                    "platformName"       : "Core",
                    "platformRootURL"    : corePlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "DataLake Platform",
                    "platformName"       : "DataLake",
                    "platformRootURL"    : dataLakePlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "Development Platform",
                    "platformName"       : "Development",
                    "platformRootURL"    : devPlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS1",
                    "description"        : "Data Lake Operations",
                    "platformName"       : "DataLake",
                    "serverName"         : "cocoMDS1"
                },
                 {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS2",
                    "description"        : "Governance",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS2"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS3",
                    "description"        : "Research",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS3"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS5",
                    "description"        : "Business Systems",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS5"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS6",
                    "description"        : "Manufacturing",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS6"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDSx",
                    "description"        : "Development",
                    "platformName"       : "Development",
                    "serverName"         : "cocoMDSx"
                },
                                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "dbisrv",
                    "description"        : "additional mds for pg",
                    "platformName"       : "Development",
                    "serverName"         : "dbisrv"
                },
                                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "intsrv",
                    "description"        : "Integration server for pg",
                    "platformName"       : "Development",
                    "serverName"         : "intsrv"
                },
            ]
        }

configureIntegrationViewService(adminPlatformURL, adminUserId, viewServerName, viewService, rexConfigBody)

# type-explorer has endpoints
viewService="tex"
texConfigBody = {
            "class":"IntegrationViewServiceConfig",
            "viewServiceAdminClass":"org.odpi.openmetadata.viewservices.tex.admin.TexViewAdmin",
            "viewServiceFullName":"Type Explorer",
            "viewServiceOperationalStatus":"ENABLED",
            "omagserverPlatformRootURL": "UNUSED",
            "omagserverName" : "UNUSED",
            "resourceEndpoints" : [
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "Core Platform",
                    "platformName"       : "Core",
                    "platformRootURL"    : corePlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "DataLake Platform",
                    "platformName"       : "DataLake",
                    "platformRootURL"    : dataLakePlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "Development Platform",
                    "platformName"       : "Development",
                    "platformRootURL"    : devPlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS1",
                    "description"        : "Data Lake Operations",
                    "platformName"       : "DataLake",
                    "serverName"         : "cocoMDS1"
                },
                 {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS2",
                    "description"        : "Governance",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS2"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS3",
                    "description"        : "Research",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS3"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS5",
                    "description"        : "Business Systems",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS5"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS6",
                    "description"        : "Manufacturing",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS6"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDSx",
                    "description"        : "Development",
                    "platformName"       : "Development",
                    "serverName"         : "cocoMDSx"
                },
                                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "dbisrv",
                    "description"        : "additional mds for pg",
                    "platformName"       : "Development",
                    "serverName"         : "dbisrv"
                },
                                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "intsrv",
                    "description"        : "Integration server for pg",
                    "platformName"       : "Development",
                    "serverName"         : "intsrv"
                },
            ]
        }

configureIntegrationViewService(adminPlatformURL, adminUserId, viewServerName, viewService, texConfigBody)

# Dino provides insight into the operational environment of egeria - this config body allows coco's platforms & servers to be accessed
viewService="dino"
DinoConfigBody = {
            "class":"IntegrationViewServiceConfig",
            "viewServiceAdminClass":"org.odpi.openmetadata.viewservices.dino.admin.DinoViewAdmin",
            "viewServiceFullName":"Dino",
            "viewServiceOperationalStatus":"ENABLED",
            "omagserverPlatformRootURL": "UNUSED",
            "omagserverName" : "UNUSED",
            "resourceEndpoints" : [
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "Core Platform",
                    "platformName"       : "Core",
                    "platformRootURL"    : corePlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "DataLake Platform",
                    "platformName"       : "DataLake",
                    "platformRootURL"    : dataLakePlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Platform",
                    "description"        : "Development Platform",
                    "platformName"       : "Development",
                    "platformRootURL"    : devPlatformURL
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS1",
                    "description"        : "Data Lake Operations",
                    "platformName"       : "DataLake",
                    "serverName"         : "cocoMDS1"
                },
                 {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS2",
                    "description"        : "Governance",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS2"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS3",
                    "description"        : "Research",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS3"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS4",
                    "description"        : "Data Lake Users",
                    "platformName"       : "DataLake",
                    "serverName"         : "cocoMDS4"
                },
                              {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS5",
                    "description"        : "Business Systems",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS5"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDS6",
                    "description"        : "Manufacturing",
                    "platformName"       : "Core",
                    "serverName"         : "cocoMDS6"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoMDSx",
                    "description"        : "Development",
                    "platformName"       : "Development",
                    "serverName"         : "cocoMDSx"
                },
                                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "dbisrv",
                    "description"        : "additional mds for pg",
                    "platformName"       : "Development",
                    "serverName"         : "dbisrv"
                },
                                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "intsrv",
                    "description"        : "integration server for pg",
                    "platformName"       : "Development",
                    "serverName"         : "intsrv"
                },
                {
                    "class"              : "ResourceEndpointConfig",
                    "resourceCategory"   : "Server",
                    "serverInstanceName" : "cocoView1",
                    "description"        : "View Server",
                    "platformName"       : "DataLake",
                    "serverName"         : "cocoView1"
                },
            ]
        }

configureIntegrationViewService(adminPlatformURL, adminUserId, viewServerName, viewService, DinoConfigBody)

deployServerToPlatform(adminPlatformURL, adminUserId, "cocoView1", lakePlatformURL)
activateServerOnPlatform("cocoView1", "dev platform", lakePlatformURL)

print ("\nDone.")



Configuring cocoView1...
   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/cocoView1/server-url-root?url=https://lab-datalake:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the maximum page size...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/max-page-size?limit=100 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... clearing the server's type...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/cocoView1/server-type?typeName= )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the server's type...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/cocoView1/server-type?typeName=View Server )
   ...... Response:  {'class': 'VoidResponse', 'relat

# Searching for the databases, tables & columns 

The databases we have found are 'Assets'

In Egeria the [OMAS](https://egeria-project.org/services/omas/?h=omas#using-the-omass) services are targetted at consuming applications, and present a higher level API than the [OMRS](https://egeria-project.org/services/omrs/?h=omrs).

To discover more information about the Assets we have imported into Postgres, we shall initially use the [Asset Owner OMAS](https://egeria-project.org/services/omas/asset-owner/overview/)

Asset Owner does not have an option to search by Asset Type, so we will need to check the types of the assets returned. To keep the search well scoped in this demo, we will further restrict the name to 'employees'. If this is changed, the later calls may need updating to reflect changed assumptions.

In [15]:
searchServer="cocoMDS1"
searchPlatformName="datalake Platform"
searchPlatformURL=dataLakePlatformURL
searchUserId="peterprofile"
searchString="employees"
searchResult = assetOwnerSearchForAssets(searchServer, searchPlatformName, searchPlatformURL, searchUserId, searchString)

Normally the function assetOwnerSearchForAssets prints the output in a tidy form,
but since we've assigned it to a variable, we need to format is specifically
here for printing.

So let's look at the full result data:

In [16]:
print (json.dumps(searchResult,indent=4))

[
    {
        "elementHeader": {
            "guid": "3c3fe575-5a1a-4cfc-999e-eab2bc39479f",
            "status": "ACTIVE",
            "type": {
                "typeId": "eab811ec-556a-45f1-9091-bc7ac8face0f",
                "typeName": "DeployedDatabaseSchema",
                "superTypeNames": [
                    "DataSet",
                    "Asset",
                    "Referenceable",
                    "OpenMetadataRoot"
                ],
                "typeVersion": 1,
                "typeDescription": "A collection of database tables and views running in a database server."
            },
            "origin": {
                "sourceServer": "cocoMDS1",
                "originCategory": "EXTERNAL_SOURCE",
                "homeMetadataCollectionId": "ce7ab29d-3c6a-41d4-b01e-d40d81019767",
                "homeMetadataCollectionName": "postgresql"
            },
            "versions": {
                "createdBy": "OMAGServer",
                "createTime": "202

## Now let's look at the database itself

In [17]:
# Find which one is the actual database 

import json
dbresult=''

for result in searchResult:
    type = result['elementHeader']['type']['typeName']
    if (type=='Database'):
        print (json.dumps(result,indent=4))
        dbresult=result
        dbguid=result['elementHeader']['guid']
        dbversion=result['elementHeader']['versions']['version']
        
# print database name and guid

print ('Found database guid: ' + dbguid + " at version: " + str(dbversion))

{
    "elementHeader": {
        "guid": "2c1e24e7-5349-455c-b99d-6f3cd93abe37",
        "status": "ACTIVE",
        "type": {
            "typeId": "0921c83f-b2db-4086-a52c-0d10e52ca078",
            "typeName": "Database",
            "superTypeNames": [
                "DataStore",
                "Asset",
                "Referenceable",
                "OpenMetadataRoot"
            ],
            "typeVersion": 2,
            "typeDescription": "A data store containing relational data."
        },
        "origin": {
            "sourceServer": "cocoMDS1",
            "originCategory": "EXTERNAL_SOURCE",
            "homeMetadataCollectionId": "ce7ab29d-3c6a-41d4-b01e-d40d81019767",
            "homeMetadataCollectionName": "postgresql"
        },
        "versions": {
            "createdBy": "OMAGServer",
            "createTime": "2022-03-08T10:18:31.330+00:00",
            "version": 1
        },
        "classifications": [
            {
                "classificationName":

Now let's get more info on that guid

## Latest change

In the above we can see a 'latest change' classification - this shows us what changes most recently, in this case a relationship. However this is just the latest change, not a audit log or list of changes.

In [18]:
# From database navigate to deployedDatabaseSchema

dbrel=dbresult['elementHeader']['classifications']
print (json.dumps(dbrel,indent=4))

[
    {
        "classificationName": "DataStoreEncoding",
        "classificationProperties": {
            "encoding": "6",
            "language": "en_US.utf-8"
        }
    },
    {
        "classificationName": "LatestChange",
        "classificationProperties": {
            "relationshipType": "DataContentForDataSet",
            "attachmentType": "DeployedDatabaseSchema",
            "changeTarget": "EntityRelationship",
            "description": "Linking Database 2c1e24e7-5349-455c-b99d-6f3cd93abe37 to DeployedDatabaseSchema e8c91720-d0cb-46a5-be3d-47e06b3b579e",
            "attachmentGUID": "e8c91720-d0cb-46a5-be3d-47e06b3b579e",
            "user": "OMAGServer",
            "changeAction": "Created"
        }
    }
]


### AssetUniverse

Retrieving the asset universe should show more about the asset and everything related.

In [19]:
# Try AssetUniverse
def assetConsumerPrintAssetUniverse(serverName, serverPlatformName, serverPlatformURL, userId, requestType):
    printAssetUniverse(serverName, serverPlatformName, serverPlatformURL, "asset-consumer", userId, requestType)
    
assetConsumerPrintAssetUniverse(searchServer, searchPlatformName, searchPlatformURL, searchUserId, dbguid)

 
GET https://lab-datalake:9443/servers/cocoMDS1/open-metadata/common-services/asset-consumer/connected-asset/users/peterprofile/assets/2c1e24e7-5349-455c-b99d-6f3cd93abe37
Returns:
{
    "class": "AssetResponse",
    "relatedHTTPCode": 200,
    "asset": {
        "class": "Asset",
        "headerVersion": 0,
        "type": {
            "class": "ElementType",
            "headerVersion": 0,
            "elementSourceServer": "cocoMDS1",
            "elementOrigin": "EXTERNAL_SOURCE",
            "elementMetadataCollectionId": "ce7ab29d-3c6a-41d4-b01e-d40d81019767",
            "elementMetadataCollectionName": "postgresql",
            "elementCreatedBy": "OMAGServer",
            "elementCreateTime": "2022-03-08T10:18:31.330+00:00",
            "elementVersion": 1,
            "elementTypeId": "0921c83f-b2db-4086-a52c-0d10e52ca078",
            "elementTypeName": "Database",
            "elementSuperTypeNames": [
                "DataStore",
                "Asset",
                

## More database information

There's limited information above -- other than 'latest change' to allow navigation to the schema. We got as far as the schema, but to navigate to the tables and columns will take quite a number of steps, so we see this OMAS isn't really appropiate beyond the initial search for a (Database) Asset.

If we used  OMRS we could follow all the low level information, but instead we'll now use a different OMAS call to get more database information including schema.

This time we'll use [Data Manager OMAS](https://egeria-project.org/services/omas/data-manager/overview/?h=data+manager+omas) which is geared to handling databases, and is also used by the PostGres Connector itself. 

There aren't yet any standard functions in the notebooks, so we'll make these calls using the generic functions.

In [30]:
dataManagerURLroot = mdrServerPlatform + '/servers/' + mdrServerName + '/open-metadata/access-services/data-manager/users/' + adminUserId 

url=dataManagerURLroot+'/databases/'+dbguid
print ('calling: ' + url)
response=issueGet(url)
printRestResponse(response)


calling: https://lab-core:9443/servers/dbisrv/open-metadata/access-services/data-manager/users/garygeeke/databases/2c1e24e7-5349-455c-b99d-6f3cd93abe37
Returns:
{
    "class": "DatabaseResponse",
    "relatedHTTPCode": 200,
    "element": {
        "elementHeader": {
            "guid": "2c1e24e7-5349-455c-b99d-6f3cd93abe37",
            "type": {
                "typeId": "0921c83f-b2db-4086-a52c-0d10e52ca078",
                "typeName": "Database",
                "superTypeNames": [
                    "DataStore",
                    "Asset",
                    "Referenceable",
                    "OpenMetadataRoot"
                ],
                "typeVersion": 2,
                "typeDescription": "A data store containing relational data."
            },
            "origin": {
                "sourceServer": "dbisrv",
                "originCategory": "EXTERNAL_SOURCE",
                "homeMetadataCollectionId": "ce7ab29d-3c6a-41d4-b01e-d40d81019767",
                "home

In [50]:
# get the schema
url=dataManagerURLroot+'/databases/'+dbguid+'/schemas?startFrom=0&pageSize=10'
print ('calling: ' + url)
response=issueGet(url)
printRestResponse(response)

calling: https://lab-core:9443/servers/dbisrv/open-metadata/access-services/data-manager/users/garygeeke/databases/2c1e24e7-5349-455c-b99d-6f3cd93abe37/schemas?startFrom=0&pageSize=10
Returns:
{
    "class": "DatabaseSchemasResponse",
    "relatedHTTPCode": 200,
    "elementList": [
        {
            "elementHeader": {
                "guid": "e8c91720-d0cb-46a5-be3d-47e06b3b579e",
                "type": {
                    "typeId": "eab811ec-556a-45f1-9091-bc7ac8face0f",
                    "typeName": "DeployedDatabaseSchema",
                    "superTypeNames": [
                        "DataSet",
                        "Asset",
                        "Referenceable",
                        "OpenMetadataRoot"
                    ],
                    "typeVersion": 1,
                    "typeDescription": "A collection of database tables and views running in a database server."
                },
                "origin": {
                    "sourceServer": "dbisrv"

In [71]:
# let's get guid of deployed database schema
schemaName=''
schemaGuid=''
elements=response.json().get('elementList')
skipList=['information_schema','pg_catalog','public']
for entry in elements:
  qn=entry['databaseSchemaProperties']['qualifiedName']
  if not any([x in qn for x in skipList]):
    schemaGuid=entry['elementHeader']['guid']
    schemaName=qn
    #if (type=='Database'):
    #    print (json.dumps(result,indent=4))
    ##    dbresult=result
    #    dbguid=result['elementHeader']['guid']
     #   dbversion=result['elementHeader']['versions']['version']
        
print('Found schema: '+ schemaName + ' with schemaGuid: ' + schemaGuid)

Found schema: employees::employees::employees with schemaGuid: e8c91720-d0cb-46a5-be3d-47e06b3b579e


In [73]:
# Now let's get the tables of the schema
url=dataManagerURLroot+'/databases/schemas/' + schemaGuid +'/tables?startFrom=0&pageSize=10'
print ('calling: ' + url)
response=issueGet(url)
printRestResponse(response)


calling: https://lab-core:9443/servers/dbisrv/open-metadata/access-services/data-manager/users/garygeeke/databases/schemas/e8c91720-d0cb-46a5-be3d-47e06b3b579e/tables?startFrom=0&pageSize=10
Returns:
{
    "class": "DatabaseTablesResponse",
    "relatedHTTPCode": 200,
    "elementList": [
        {
            "elementHeader": {
                "guid": "2f8c2a8b-77fb-4053-9d1d-4f3c5ad00a9b",
                "type": {
                    "typeId": "ce7e72b8-396a-4013-8688-f9d973067425",
                    "typeName": "RelationalTable",
                    "superTypeNames": [
                        "SchemaAttribute",
                        "SchemaElement",
                        "Referenceable",
                        "OpenMetadataRoot"
                    ],
                    "typeVersion": 1,
                    "typeDescription": "A table within a relational database schema type."
                },
                "origin": {
                    "sourceServer": "dbisrv",
     

In [79]:
# Now let's go through the list of tables and pull out the important info

etabguid=''
elements=response.json().get('elementList')

for entry in elements:
  print (entry['elementHeader']['guid'] + ' ' + entry ['databaseTableProperties']['displayName'])
  if entry['databaseTableProperties']['displayName'] == 'employee':
    etabguid=entry['elementHeader']['guid']
    
print ('Also saving guid for the employee table so we can look at columns -- ' + etabguid)

  

2f8c2a8b-77fb-4053-9d1d-4f3c5ad00a9b department_manager
c003d113-7cd1-4d96-a903-c4e3ea008143 department_employee
74ab6fb0-1764-4337-9c78-79f7476f8cf1 department
232b4dc7-ccf3-4892-94a4-2517fb457c52 title
3a4f8cc5-60ba-4c2a-b477-4889960fbfe6 salary
dd770fd4-0054-4eed-9129-57efbbcae556 employee
13778272-9ee5-49c7-ae89-65fb8cf63b7a test3
Also saving guid for the employee table so we can look at columns -- dd770fd4-0054-4eed-9129-57efbbcae556


In [81]:
# Now let's get the columns of the table
url=dataManagerURLroot+'/databases/schemas/tables/' + etabguid +'/columns?startFrom=0&pageSize=10'
print ('calling: ' + url)
response=issueGet(url)
printRestResponse(response)

calling: https://lab-core:9443/servers/dbisrv/open-metadata/access-services/data-manager/users/garygeeke/databases/schemas/tables/dd770fd4-0054-4eed-9129-57efbbcae556/columns?startFrom=0&pageSize=10
Returns:
{
    "class": "DatabaseColumnsResponse",
    "relatedHTTPCode": 200,
    "elementList": [
        {
            "elementHeader": {
                "guid": "91edf997-c865-4927-8121-b382b2042e2e",
                "type": {
                    "typeId": "aa8d5470-6dbc-4648-9e2f-045e5df9d2f9",
                    "typeName": "RelationalColumn",
                    "superTypeNames": [
                        "TabularColumn",
                        "SchemaAttribute",
                        "SchemaElement",
                        "Referenceable",
                        "OpenMetadataRoot"
                    ],
                    "typeVersion": 2,
                    "typeDescription": "A column within a relational table."
                },
                "origin": {
              

In [82]:
# Now let's go through the list of columns and pull out the important info


elements=response.json().get('elementList')

for entry in elements:
  print (entry['elementHeader']['guid'] + ' ' + entry ['databaseColumnProperties']['displayName'])




91edf997-c865-4927-8121-b382b2042e2e last_name
957e6442-d7c7-47e8-867a-9cdaf4fb1ad4 hire_date
4d91e428-2073-441a-b4d3-e829d586439d gender
71bf94cc-e44b-4ca0-850b-e4ba2aafdbcf id
e37a88d4-3836-41f9-ba69-afd44d8d1ed4 birth_date
179de31f-7f01-4873-b5c5-3dbd30b7861d first_name


In [ ]:
# Next we should define some standard functions for retrieving this kind of information

## Restarting the integration connector

This will restart the integration connector, and force it to rescan the postgres database


In [ ]:
# Restart integration connector
restartIntegrationConnector(intServerName, intServerPlatformName, intServerPlatformURL, adminUserId, integrationServiceURLMarker, "postgresql")

# Not working -- saved content

Below follows some queries/calls that currently aren't particularly useful, but might be in future


In [ ]:
## Experimental area

searchServer="cocoMDS2"
searchPlatformName="core Platform"
searchPlatformURL=corePlatformURL
searchUserId="garygeeke"
searchString=".*"
searchForAssets(searchServer, searchPlatformName, searchPlatformURL, searchUserId, searchString)

assetConsumerPrintAssets(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, searchUserId, searchString)




## Summary

We've learnt more about how to pull in metadata from another source using an integration connector.

Please join the community & help us with feedback, ideas, contributions.